In [175]:
import cv2 as cv
print("Current version of openCV=", cv.__version__)
import numpy as np
import os
import glob

Current version of openCV= 4.1.1


In [176]:
left_calibration_path = './calibration/fisheye_calibrated/{}.png'
right_calibration_path ='./calibration/pinhole_calibrated/{}.png'

In [177]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
points_per_row = 6
points_per_column = 10
objp = np.zeros((points_per_column*points_per_row,3), np.float32)
objp[:,:2] = np.mgrid[0:points_per_row,0:points_per_column].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpointsL = [] # 2d points in Right image plane.
imgpointsR = [] # 2d points in Left image plane.
imagesL = glob.glob(left_calibration_path.format('*'))
imagesR = glob.glob(right_calibration_path.format('*'))
for img_id in range(len(imagesR)):
    fnameL = left_calibration_path.format(img_id)
    fnameR = right_calibration_path.format(img_id)
    imgL = cv.imread(fnameL)
    imgR = cv.imread(fnameR)
    grayL = cv.cvtColor(imgL, cv.COLOR_BGR2GRAY)
    grayR = cv.cvtColor(imgR, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    retL, cornersL = cv.findChessboardCorners(grayL, (points_per_row,points_per_column), None)
    retR, cornersR = cv.findChessboardCorners(grayR, (points_per_row,points_per_column), None)
    # If found on both, add object points, image points (after refining them)
    if retL == True and retR == True:
        objpoints.append(objp)
        corners2L = cv.cornerSubPix(grayL,cornersL, (11,11), (-1,-1), criteria)
        corners2R = cv.cornerSubPix(grayR,cornersR, (11,11), (-1,-1), criteria)
        imgpointsL.append(cornersL)
        imgpointsR.append(cornersR)
        # Draw and display the corners
        cv.drawChessboardCorners(imgL, (points_per_row,points_per_column), corners2L, retL)
        cv.drawChessboardCorners(imgR, (points_per_row,points_per_column), corners2R, retR)
        cv.imshow('img', imgL)
        cv.waitKey(500)
        cv.imshow('img', imgR)
        cv.waitKey(500)
cv.destroyAllWindows()

In [287]:
img = cv.imread(left_calibration_path.format(img_id))
width, height = img.shape[:-1]
R = None
T = None
E = None
F = None
stereocalib_criteria = (cv.TERM_CRITERIA_MAX_ITER | cv.TERM_CRITERIA_EPS, 100, 1e-5)
#This flag decreases the accuracy but now the results match better... it is probably better for the 3d reconstruct to be on
stereocalib_flags = cv.CALIB_RATIONAL_MODEL | \
cv.CALIB_SAME_FOCAL_LENGTH | \
cv.CALIB_ZERO_TANGENT_DIST 
cv.CALIB_FIX_ASPECT_RATIO 
cv.CALIB_FIX_K3 | \
cv.CALIB_FIX_K4 | \
cv.CALIB_FIX_K5 
print(width, height)
retval, KL, DL, KR, DR, R, T, E, F = \
 cv.stereoCalibrate(objpoints, imgpointsL, imgpointsR,
                                #imageSize = (width, height),
                                imageSize = (height, width),
                                cameraMatrix1 = None,
                                distCoeffs1 = None,
                                cameraMatrix2 = None,
                                distCoeffs2 = None,
                                R=R,
                                T=T,
                                E=E,
                                F=F,
                                #flags=cv.CALIB_FIX_INTRINSIC
                                flags=stereocalib_flags,
                                criteria=stereocalib_criteria
                   )

720 1280


In [288]:
print(retval)
print("cameraM1: \n", KL)
print("distCoeffs1: \n", DL)
print("cameraM2: \n", KR)
print("distCoeffs2: \n", DR)
print("R: ", R)
print("T: ", T)
print("E: ", E)
print("F: ", F)

3.1991655665973475
cameraM1: 
 [[797.78882436   0.         718.40941085]
 [  0.         796.53894865 393.57071754]
 [  0.           0.           1.        ]]
distCoeffs1: 
 [[ -6.20343413  22.07957474   0.           0.         -14.63131817
   -6.17944726  22.06311045 -14.56434341   0.           0.
    0.           0.           0.           0.        ]]
cameraM2: 
 [[797.78882436   0.         628.22801221]
 [  0.         796.53894865 339.19631041]
 [  0.           0.           1.        ]]
distCoeffs2: 
 [[107.85465794 878.13274551   0.           0.         469.29840536
   90.49919797 794.54394693 409.58038284   0.           0.
    0.           0.           0.           0.        ]]
R:  [[ 0.99998129 -0.00367373  0.00489189]
 [ 0.00364551  0.99997673  0.00576601]
 [-0.00491295 -0.00574806  0.99997141]]
T:  [[ 3.64423844]
 [-0.08601046]
 [-0.01266609]]
E:  [[ 4.68739856e-04  1.31601902e-02 -8.59349731e-02]
 [ 5.23812515e-03  2.09938475e-02 -3.64419622e+00]
 [ 9.92939495e-02  3.64383767e+

In [277]:
img_size = imgL.shape[:2]
alpha = 0 ### =0 crop, =1 keep dimensions
print(width, height)

RL = np.zeros(shape=(3,3))
RR = np.zeros(shape=(3,3))
PL = np.zeros(shape=(3,4))
PR = np.zeros(shape=(3,4))

"""out = cv.stereoRectify(KL, DL, KR, DR, (width, height), R=R, T=T, 
                       R1=RL, R2=RR, P1=PL, P2=PR,
                      Q=None, flags=cv.CALIB_ZERO_DISPARITY, alpha=alpha, newImageSize=(0,0))
"""
RL, RR, PL, PR, Q, roiL, roiR = cv.stereoRectify(KL, 
                                                 DL, 
                                                 KR, 
                                                 DR, 
                                                 (height, width), 
                                                 R=R, 
                                                 T=T, 
                                                 alpha=alpha)

720 1280


In [278]:
print("RL: \n", RL)
print("RR: \n", RR)
print("PL: \n", PL)
print("RR: \n", PR)
print("Q: ", Q)
print("roiL: \n", roiL)
print("roiR: \n", roiR)


RL: 
 [[ 0.99962791 -0.02724722  0.00127989]
 [ 0.02724338  0.99962454  0.00292818]
 [-0.00135919 -0.00289222  0.99999489]]
RR: 
 [[ 0.99971556 -0.02359505 -0.00347466]
 [ 0.02358484  0.99971748 -0.00295166]
 [ 0.00354332  0.00286887  0.99998961]]
PL: 
 [[1.01729403e+03 0.00000000e+00 6.85404419e+02 0.00000000e+00]
 [0.00000000e+00 1.01729403e+03 3.70340416e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]]
RR: 
 [[1.01729403e+03 0.00000000e+00 6.85404419e+02 3.70831682e+03]
 [0.00000000e+00 1.01729403e+03 3.70340416e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]]
Q:  [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -6.85404419e+02]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -3.70340416e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.01729403e+03]
 [ 0.00000000e+00  0.00000000e+00 -2.74327703e-01  0.00000000e+00]]
roiL: 
 (0, 0, 1280, 720)
roiR: 
 (0, 0, 1280, 720)


In [279]:
print("Applying Undistort\n")
#map1x, map1y = cv.initUndistortRectifyMap(KL, DL, RL, PL, (width, height), cv.CV_32FC1)
#map2x, map2y = cv.initUndistortRectifyMap(KR, DR, RR, PR, (width, height), cv.CV_32FC1)

map1x, map1y = cv.initUndistortRectifyMap(KL, DL, RL, PL, (height, width), cv.CV_32FC1)
map2x, map2y = cv.initUndistortRectifyMap(KR, DR, RR, PR, (height, width), cv.CV_32FC1)
print("Undistort complete\n")

"""
for img_id in range(len(imagesR)):
    fnameL = left_calibration_path.format(img_id)
    fnameR = right_calibration_path.format(img_id)
    print("fname: ", fnameL, fnameR)
    imgL = cv.imread(fnameL)
    imgR = cv.imread(fnameR)
    imgU1 = np.zeros((height,width,3), np.uint8)
    imgU1 = cv.remap(imgL, map1x, map1y, cv.INTER_LINEAR, imgU1, cv.BORDER_CONSTANT, 0)
    imgU2 = cv.remap(imgR, map2x, map2y, cv.INTER_LINEAR)
    cv.imshow("imageL", imgL);
    cv.imshow("imageR", imgR);
    cv.imshow("image1L", imgU1);
    cv.imshow("image2R", imgU2);
    k = cv.waitKey(5);
    if(k==27):
        break;
"""
print("Maps1: \n", map1x, map1y)
print("Maps2: \n", map2x, map2y)

#image remap example
img_id = 10
fnameL = left_calibration_path.format(img_id)
fnameR = right_calibration_path.format(img_id)
print("fname: ", fnameL, fnameR)
imgL = cv.imread(fnameL)
imgR = cv.imread(fnameR)
cv.imshow('imgL', imgL)
cv.waitKey(500)

left_img_remap = cv.remap(imgL, map1x, map1y, cv.INTER_LANCZOS4)
right_img_remap = cv.remap(imgR, map2x, map2y, cv.INTER_LANCZOS4)

Applying Undistort

Undistort complete

Maps1: 
 [[ 177.00061  177.76949  178.53848 ... 1168.777   1169.5513  1170.3256 ]
 [ 177.01857  177.7875   178.55655 ... 1168.7977  1169.572   1170.3463 ]
 [ 177.03658  177.80556  178.57466 ... 1168.8184  1169.5928  1170.3672 ]
 ...
 [ 193.13321  193.90231  194.67152 ... 1183.0083  1183.7819  1184.5554 ]
 [ 193.1594   193.92847  194.69763 ... 1183.0277  1183.8013  1184.5747 ]
 [ 193.18562  193.95464  194.72375 ... 1183.0471  1183.8207  1184.5941 ]] [[118.51232  118.4871   118.461914 ...  91.733     91.7132    91.693436]
 [119.286385 119.26119  119.236046 ...  92.5084    92.48858   92.46879 ]
 [120.06047  120.03532  120.0102   ...  93.283806  93.26397   93.244156]
 ...
 [674.1691   674.1514   674.1337   ... 646.98     646.9582   646.9364  ]
 [674.9407   674.92303  674.90533  ... 647.75256  647.73083  647.70905 ]
 [675.7122   675.6946   675.67694  ... 648.5252   648.5034   648.48157 ]]
Maps2: 
 [[  21.55909    22.43064    23.302172 ... 1146.185    

In [282]:
cv.imshow("imgL remap", left_img_remap)
cv.imshow("imgR remap", right_img_remap)
cv.waitKey(500)
#cv.destroyAllWindows()